In [5]:
import torch
import numpy as np

%load_ext autoreload
%autoreload 2

# extract activation map for ISIC 2017 validation set

# compute culprit score from the activations, predictions, and gt

In [ ]:
from culprit import *
import json
config = json.load(open('config_skin_alexnet.json'))
os.environ["CUDA_VISIBLE_DEVICES"]= '1'
resume = 'skinmodel/checkpoint.pth'

In [6]:
from uncertainty import *

In [47]:
experiment_saved_path = './saved'
uncty = Uncertainty(experiment_saved_path)

*** actv shape (ignore dim 0 - batch size) is: [torch.Size([64, 64, 55, 55]), torch.Size([64, 192, 27, 27]), torch.Size([64, 384, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 256, 13, 13]), torch.Size([64, 4096]), torch.Size([64, 4096]), torch.Size([64, 2])] .
*** data loaded ***
*** Flattened actv vector shape is torch.Size([150, 9346]).








  0%|          | 0/10 [00:00<?, ?it/s]

TypeError: Traceback (most recent call last):
  File "/local-scratch/weinaj/anaconda3/envs/pt10/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/local-scratch/weinaj/anaconda3/envs/pt10/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 138, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/local-scratch/weinaj/pytorch_classification_template/data_loader/data_loaders.py", line 29, in __getitem__
    sample = self.transform(sample)
  File "/local-scratch/weinaj/anaconda3/envs/pt10/lib/python3.6/site-packages/torchvision/transforms/transforms.py", line 49, in __call__
    img = t(img)
  File "/local-scratch/weinaj/pytorch_classification_template/data_loader/img_transform.py", line 27, in __call__
    img_seg = trF.resize(image_seg, self.size, Image.NEAREST)
  File "/local-scratch/weinaj/anaconda3/envs/pt10/lib/python3.6/site-packag

In [19]:
a = None
print(a is None)

True


In [147]:
X = np.ones((3,2))
Y = np.ones((3,2))+1

In [148]:
pairwise_distances(X,Y)

array([[1.41421356, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356],
       [1.41421356, 1.41421356, 1.41421356]])